In [2]:
import pandas as pd

from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

In [3]:
training = pd.read_csv("training.csv")

In [48]:
training

,Unnamed: 0,sessions,Inflation Rate,trials,resources,library,signups,hootsuite: (United States),hubspot: (United States),sprout social: (United States),free_mau_lag,paid_mau_lag
0,1,3187592,7.1,24858,320077,509679,64199,9.25,60.75,5.25,58067.416667,22326.833333
1,2,2843479,6.5,26109,245381,401437,63090,8.00,54.50,4.50,97188.633333,29519.366667
2,3,4082166,6.4,30089,258566,430695,82169,10.20,75.60,5.40,102737.967742,29925.967742
3,4,3255178,6.0,27220,285808,300683,72377,9.75,72.00,5.50,104090.451613,29510.516129
4,5,3420256,5.0,29823,372715,396321,80921,10.00,68.00,5.25,107603.464286,29828.714286
5,6,4041962,4.9,28478,235262,369495,75224,9.00,69.60,5.20,109453.870968,30106.032258
6,7,3204193,4.0,29676,254082,397158,78185,8.50,69.25,5.25,109917.300000,30072.166667
7,8,3256314,3.0,29009,260770,347053,79726,8.75,67.75,5.00,108269.935484,29937.225806
8,9,4023641,3.2,27772,261318,359864,81583,7.80,67.60,5.00,108339.366667,29733.733333
9,10,3160092,3.7,28855,249522,351905,84007,7.00,67.25,5.00,108558.645161,29299.548387


In [49]:
# Drop the 'Unnamed: 0' column from training dataframe since it's just an index
training = training.drop('Unnamed: 0', axis=1)

In [97]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(training)


mod = CTGANSynthesizer(metadata, epochs = 500)

/Users/rvrah/Documents/bufferFinanceForecast/.venv/lib/python3.9/site-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/Users/rvrah/Documents/bufferFinanceForecast/.venv/lib/python3.9/site-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [98]:
mod.fit(training)


In [99]:
test = mod.sample(500)

In [100]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=training,
    synthetic_data=test,
    metadata=metadata)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 11/11 [00:00<00:00, 747.90it/s]|
Column Shapes Score: 78.4%

(2/2) Evaluating Column Pair Trends: |██████████| 55/55 [00:00<00:00, 1001.05it/s]|
Column Pair Trends Score: 81.65%

Overall Score (Average): 80.03%



In [101]:
mod.save(
    filepath='synthetic.pkl'
)

In [55]:
import joblib

In [114]:
import pickle
from sklearn.linear_model import LinearRegression
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
import pandas as pd
import numpy as np


#to load the model
def load_model() -> LinearRegression:
    with open("lr_model.pkl", 'rb') as file:
        data = joblib.load(file)
    
    return data
    


def prediction(model: LinearRegression, synthData: CTGANSynthesizer, params: dict = None, sample_size:int = 200):
    newData = synthData.sample(sample_size)
    if params is not None:
        for x in params:
            newData[x] = params[x]
    print(newData)
    
    result = model.predict(newData)
    return np.mean(result), np.std(result)

def load_synthetic() -> CTGANSynthesizer:
    return CTGANSynthesizer.load(filepath='synthetic.pkl')





In [115]:
rf = load_model()
gen = load_synthetic()

In [116]:
type(rf)

sklearn.linear_model._base.LinearRegression

In [124]:
params = {"trials": 80000}

In [125]:
prediction(rf, gen, params)

     sessions  Inflation Rate  trials  resources  library  signups  \
0     6374203             2.8   80000     614193   567189   117785   
1     9285476             3.2   80000     456954   335914    42023   
2    13951484             3.3   80000     569040   720591    82685   
3    15918716             3.4   80000     328152   332799    74172   
4     3810546             2.6   80000     176368   287321    13071   
..        ...             ...     ...        ...      ...      ...   
195  15228647             3.4   80000     632703   892131   542797   
196   2622644             4.8   80000     466483   106282   875941   
197   9387637             3.4   80000     482942   199706    55844   
198   9019820             2.8   80000     672956   959943    59820   
199   7787082             3.4   80000     918232   595005    72648   

     hootsuite: (United States)  hubspot: (United States)  \
0                          5.92                     65.59   
1                          5.46      

(np.float64(33930.882746288014), np.float64(7293.9988228820985))